### Darcy Advect-diffuse net

Implement the MR-AIV pipeline in [J. D. Toscano et al. 2025](https://doi.org/10.1101/2025.07.30.667741) using pytorch + lightning.

In [ ]:
# First choose a config file to apply
config_path = "configs/dcpinn_full_runtest.yaml"

#### 1. Data preparation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

torch.set_float32_matmul_precision('medium')

from modules.dc_net import AD_DC_Net
from modules.data_module import CharacteristicDomain, DCEMRIDataModule, VelocityDataModule, PermeabilityDataModule
from utils.config_loader import Train_Config
from utils.train_wrapper import train_all_phases
from utils.visualize import draw_nifti_slices_with_time, draw_nifti_slices_with_threshold, interactive_quiver
from utils.io import load_dcemri_data, save_velocity_mat, load_DTI
%matplotlib widget
%load_ext autoreload
%autoreload 2

# load cfg from yaml path.
cfg = Train_Config(config_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:

data,mask,pixdim,x,y,z,t = load_dcemri_data(cfg.dcemrinp_data_path)

char_domain = CharacteristicDomain(data.shape, mask, t, pixdim, device)

print("L_star: ",char_domain.L_star, "T_star: ", char_domain.T_star)
# batch_size is sum over data's point
c_dataset = DCEMRIDataModule(data, char_domain,  
                           batch_size=int(mask.sum()*len(t)), num_workers=cfg.dataset_num_workers, device=device)

# set up to get num_train_points
c_dataset.setup()
print("num_train_points: ", c_dataset.num_train_points, "batch_size: ", c_dataset.batch_size)

In [ ]:
if cfg.use_DTI:
    DTI_tensor,DTI_MD = load_DTI(char_domain, "data/DCE_nii_data/dti_tensor_3_3.mat", data.shape[:3])
else:
    DTI_tensor = None

char_domain.set_DTI_or_coef(DTI_tensor if cfg.use_DTI else 2.4e-4)
print("Pe_g: ", char_domain.Pe_g)

In [ ]:
if cfg.use_DTI:
    draw_nifti_slices_with_threshold(DTI_MD, brain_mask=mask)

In [ ]:
# use front-tracking to get initial velocity field.
from utils.velocity_guess import front_tracking_velocity
# shape (nx, ny, nz, 3), only use half of the timestep to training, consistant with c_dataset

initial_velocity_field = front_tracking_velocity(data[:,:,:,::2], 
                                                 dt=t[2] - t[0])
# scaling, front tracking give unit in grid/min, we need to scale to characteristic velocity
initial_velocity_field *= (char_domain.pixdim / char_domain.V_star)

# draw initial velocity magnitude
vel_mag = np.linalg.norm(initial_velocity_field, axis=-1)
# print(pixdim, char_domain.domain_shape, char_domain.L_star, char_domain.T_star, char_domain.V_star)
print("velocity max and min: ", vel_mag.max(), vel_mag.min())
draw_nifti_slices_with_threshold(vel_mag)

v_dataset = VelocityDataModule(initial_velocity_field, char_domain,
                              batch_size=int(mask.sum()), num_workers=cfg.dataset_num_workers, device=device)
# set up to get num_train_points
v_dataset.setup()

In [ ]:
interactive_quiver(initial_velocity_field[:,:,:,0], initial_velocity_field[:,:,:,1], 
                   initial_velocity_field[:,:,:,2], char_domain.pixdim,default_elev=-62.76, default_azim=-10.87)


In [ ]:
from utils.permeability_guess import estimate_initial_permeability

initial_permeability, smooth_permeability = estimate_initial_permeability(data, t, ser_threshold=12.0,
                                                     time_threshold_min=t[4])
print("Initial permeability max and min: ", smooth_permeability.max(), smooth_permeability.min())
draw_nifti_slices_with_threshold(smooth_permeability, brain_mask=mask)
# prepare permeability datamodule, according to the paper,
# "smooth" initial permeability guess is almost certainly the better and safer choice for initializing the NN_k network
k_dataset = PermeabilityDataModule(smooth_permeability, char_domain,
                                  batch_size=int(mask.sum()), num_workers=cfg.dataset_num_workers, device=device)


#### 2. Network definition


In [ ]:
from modules.c_trainer import CNet_Init, CNet_DenoiseInit

from modules.dc_trainer import DCPINN_InitK, DCPINN_InitP, DCPINN_ADPDE_P, DCPINN_ADPDE_P_K, DCPINN_Joint

# write a trainner getter to factorize.

def DCPINN_trainer_getter(train_phase, ad_dc_net, phase_cfg=None):
    if train_phase == CNet_Init.train_phase:
        pinn_model = CNet_Init(ad_dc_net.c_net, c_dataset.num_train_points)
        datamodule = c_dataset
    elif train_phase == CNet_DenoiseInit.train_phase:
        pinn_model = CNet_DenoiseInit(ad_dc_net.c_net, c_dataset.num_train_points)
        datamodule = c_dataset
    elif train_phase == DCPINN_InitK.train_phase:
        pinn_model = DCPINN_InitK(ad_dc_net, k_dataset.num_train_points)
        datamodule = k_dataset
    elif train_phase == DCPINN_InitP.train_phase:
        pinn_model = DCPINN_InitP(ad_dc_net, v_dataset.num_train_points)
        datamodule = v_dataset
    elif train_phase == DCPINN_ADPDE_P.train_phase:
        pinn_model = DCPINN_ADPDE_P(ad_dc_net, c_dataset.num_train_points)
        datamodule = c_dataset
    elif train_phase == DCPINN_ADPDE_P_K.train_phase:
        pinn_model = DCPINN_ADPDE_P_K(ad_dc_net, c_dataset.num_train_points)
        datamodule = c_dataset
    elif train_phase == DCPINN_Joint.train_phase:
        pinn_model = DCPINN_Joint(ad_dc_net, c_dataset.num_train_points)
        datamodule = c_dataset
    else:
        raise ValueError(f"Unknown train_phase {train_phase}")
    return pinn_model, datamodule


In [ ]:
P_star = np.mean(char_domain.V_star * cfg.viscosity * char_domain.L_star / k_dataset.K_star)
print("P_star: ", P_star)

#### 3. Training step

- first we has initializaiton for c and v net
- then pde loss for only v net
- finally joint optimization.

In [ ]:
# Instantiate the model and trainer or load from checkpoint:
ad_dc_net = AD_DC_Net(c_layers=[4] + [cfg.neuron_num for _ in range(cfg.hid_layer_num)] + [1],
        k_layers=[3] + [cfg.neuron_num for _ in range(cfg.hid_layer_num)] + [1], # if anisotropic, change to 3 output
        p_layers=[3] + [cfg.neuron_num for _ in range(cfg.hid_layer_num)] + [1],
        data=data, char_domain=char_domain, C_star=c_dataset.C_star,
        K_star=k_dataset.K_star, P_star=P_star,
        positional_encoding=cfg.use_positional_encoding,
        freq_nums=cfg.positional_encode_nums,
        gamma_space=cfg.position_encode_freq_scale,
        anisotropic=False)

if cfg.do_training:
    pinn_model = train_all_phases(ad_dc_net, DCPINN_trainer_getter, cfg)
else:
    checkpoint = torch.load(cfg.ckpt_path, map_location="cpu")
    if "state_dict" in checkpoint:
        pinn_model = DCPINN_trainer_getter(checkpoint.get("train_phase"), ad_dc_net)[0]
        pinn_model.load_state_dict(checkpoint['state_dict'], strict=True)
    else:
        pinn_model = DCPINN_Joint(ad_dc_net, c_dataset.num_train_points)
        ad_dc_net.load_state_dict(checkpoint['state_dict'], strict=False)


# After training, visualize the learned velocity field and diffusivity
# Extract learned parameters
D_learned = pinn_model.ad_dc_net.D.item()

print(f"Learned diffusivity D: {D_learned}")

#### 4. Visualize result.
Finally we will save the velocity mat to run GLAD (get pathline figure) in matlab.

In [ ]:
# slice smooth_permeability and draw it
from utils.visualize import draw_colorful_slice_image
val_slice_z = [char_domain.domain_shape[2] // 2 - 6, char_domain.domain_shape[2] // 2, char_domain.domain_shape[2] // 2 + 6]
slices = smooth_permeability[:,:,val_slice_z]
vis_k_list = []
for i in range(slices.shape[2]):
    vis_k = draw_colorful_slice_image(slices[:,:,i], cmap='cool', mask=char_domain.mask[:,:,val_slice_z[i]])
    vis_k_list.append(vis_k)
# draw learned permeability
k_fig = np.concatenate(vis_k_list, axis=1)
plt.figure(figsize=(8,3))
plt.imshow(k_fig)

In [ ]:
# first visualize p and k fields
p_fig = pinn_model.ad_dc_net.v_dc_net.p_net.draw_pressure_slices()
plt.figure(figsize=(8,3))
plt.imshow(p_fig)
k_fig = pinn_model.ad_dc_net.v_dc_net.k_net.draw_permeability_volume()
plt.figure(figsize=(8,3))
plt.imshow(k_fig)

In [ ]:

# 1. Extract learned velocity field on a grid for visualization
pinn_model.to(device)
# using mask to filter out the background vectors
# _ = interactive_quiver(vx, vy, vz, pixdim, default_elev=-62.76, default_azim=-10.87)
nx, ny, nz = data.shape[0], data.shape[1], data.shape[2]
v_fig, vx,vy,vz = pinn_model.ad_dc_net.v_dc_net.draw_velocity_volume()
# change vx,vy,vz from characteristic to mm/min then cell/min
vx = vx * (char_domain.V_star[0] / char_domain.pixdim[0])
vy = vy * (char_domain.V_star[1] / char_domain.pixdim[1])
vz = vz * (char_domain.V_star[2] / char_domain.pixdim[2])
print(v_fig.shape, "\nvx_min: ", vx.min(), "\tvx_max:", vx.max(),
      "\nvy_min: ", vy.min(), "\tvy_max:", vy.max(),
      "\nvz_min: ", vz.min(), "\tvz_max:", vz.max())
plt.figure(figsize=(8,8))
plt.imshow(v_fig)
# use a grid (from real to characteristic) to extract velocity field

In [ ]:
# 2. Extract all density field with x,y,z,t 4D grid for visualization

grid_tensor_4d = char_domain.get_characteristic_geotimedomain()  # include time part
print(grid_tensor_4d.shape)
def predict_concentration_4d(c_net, pts4, nx, ny, nz, nt, batch_size=200_000):
    out_list = []
    with torch.no_grad():
        for i in range(0, pts4.shape[0], batch_size):
            chunk = pts4[i:i+batch_size]
            pred = c_net(chunk)
            out_list.append(pred.cpu())
    C_flat = torch.cat(out_list, dim=0).numpy().reshape(nx, ny, nz, nt)
    # C_flat = torch.cat(out_list, dim=0).numpy().reshape(nx, ny, nz, nt)
    return C_flat

C_pred_4d = predict_concentration_4d(pinn_model.ad_dc_net.c_net, grid_tensor_4d, nx, ny, nz, len(t), batch_size=200_000)

# Back to physical units
C_pred_4d *= c_dataset.C_star
print("Pred shape:", C_pred_4d.shape, "True shape:", data.shape)  # (nx, ny, nz, nt)

_ = draw_nifti_slices_with_time(C_pred_4d, data, mask)
# try calculate loss with true data
# from sklearn.metrics import mean_squared_error
# mse_overall = mean_squared_error(data.flatten(), C_pred_4d.flatten())


In [ ]:
# Export velocity to .mat format so that we could run GLAD using matlab
# the rOMT code require velocity reshaped to N*3 x 1, with unit cell/min
save_velocity_mat(vx,vy,vz,pixdim, 
                  D=pinn_model.ad_dc_net.D.item() if not cfg.use_DTI else pinn_model.ad_dc_net.D_normalized.item(), 
                  use_DTI=cfg.use_DTI,path=f"{cfg.result_folder}/predict_velocity.mat")